=============================================UNIÓN INTERNA=================================

In [21]:
%%writefile unionInterna.py
#!/usr/bin/env python
from mrjob.job import MRJob
import re,sys,os

class unionInterna(MRJob):
    
    #Función que limpia lo que el parámetro devuelve en forma "file://nombre_fichero" 
    #para dejarlo solo con el nombre del fichero: "nombre_fichero"
    def limpiarNombreArchivo(self,archivo):
        encontradaBarra=False
        tamano=len(archivo)
        posicion=tamano-1
        while encontradaBarra==False or posicion==0:
            if archivo[posicion]=="/":
                encontradaBarra=True
                return archivo[posicion+1:tamano]
            else:
                posicion-=1
        if posicion==0:
            return archivo
    
    def mapper_init(self):
        self.namefile=self.limpiarNombreArchivo(os.getenv('map_input_file')) 
        #Usamos el parámetro para saber que nos llega desde streaming
        #map_input_file devuelve una cadena de caracteres correspondiente al 
        #archivo de entrada por el stream en formato: file://nombre_fichero
        
    def mapper(self,_,line):
        clave=""
        linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            if self.namefile=="tablaA.csv":
                linea.append(self.namefile)#Añadimos al registro el nombre del archivo
                clave=linea[0] #Esta clave es la común de la tabla 1, que me permite hacer la unión con la de la tabla 2 
                yield clave,linea
            else:
                linea.append(self.namefile) #Aquí estoy usando el nombre del archivo, pero puedo usar otro identificador
                                        #como en el libro Dessign Pattern que usa A y B. Añadimos al registro el nombre
                                        #del archivo.
                clave=linea[0] #Esta es la otra clave común de la tabla 2
                yield clave,linea
        
    def reducer(self,key,values):
        listaA=[]
        listaB=[]
        #Llenamos las dos listas
        for valor in values:
            if valor[len(valor)-1]=="tablaA.csv":
                listaA.append(valor)
            else:
                listaB.append(valor)
        
        # Union interna ########################
        if listaA and listaB:
            for A in listaA:
                for B in listaB:
                    yield key,(A, B)
if __name__ == '__main__':
    unionInterna.run()

Overwriting unionInterna.py


In [ ]:
!python unionInterna.py archivos_datos/tablaA.csv archivos_datos/tablaB.csv